# Gurobi Exploration

In [19]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import sys
import math
import random
from itertools import permutations

In [25]:
#initializing model
m = gp.Model("weekly_tote_distribution") 

#loading data 
TruckPoints = pd.read_csv('../data/truck_service_pts_galv.csv')

In [15]:
TruckPoints.head()

,Name,Longitude,Latitude,Daily_Pickup_Totes,Weekly_Dropoff_Totes,user_ratings_total,category,rating,weekly_high,weekly_low,best_weekly_estimate,Address,geometry,Adoption,Adoption Guess,Visited,Influence,location_type,pickup_type,id
0,Moody Gardens,-94.852300,29.273600,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,depot,Truck,NaN
1,Sugar Bean,-94.870418,29.261029,1,1,476.0,cafe,4.7,200,50,200,"11 Evia Main, Galveston",POINT (-94.8704183 29.2610286),Curious,High,No,High,indoor,Truck,NaN
2,MAC-IES,-94.791385,29.298912,1,1,73.0,restaurant,5.0,1500,1050,1500,"1110 23rd Street, Galveston",POINT (-94.7913851 29.2989118),Curious,High,No,Low,indoor,Truck,NaN
3,Good Dough,-94.808531,29.290612,1,1,68.0,bakery,4.6,70,35,70,"1508 39th Street, Galveston",POINT (-94.808531 29.2906119),Curious,High,No,Med,indoor,Truck,NaN
4,Porch Cafe,-94.750192,29.319581,1,1,332.0,restaurant,4.1,300,NaN,100,"1625 East Beach Drive #2A, Galveston",POINT (-94.7501916 29.3195806),Curious,High,No,Low,indoor,Truck,NaN


In [13]:
#splitting by indoor outdoor

DropoffOnly = TruckPoints.copy()
DropoffOnly = DropoffOnly[DropoffOnly['Weekly_Dropoff_Totes'] >= 1]
DropoffOnly

,Name,Longitude,Latitude,Daily_Pickup_Totes,Weekly_Dropoff_Totes,user_ratings_total,category,rating,weekly_high,weekly_low,best_weekly_estimate,Address,geometry,Adoption,Adoption Guess,Visited,Influence,location_type,pickup_type,id
1,Sugar Bean,-94.870418,29.261029,1,1,476.0,cafe,4.7,200,50,200,"11 Evia Main, Galveston",POINT (-94.8704183 29.2610286),Curious,High,No,High,indoor,Truck,NaN
2,MAC-IES,-94.791385,29.298912,1,1,73.0,restaurant,5.0,1500,1050,1500,"1110 23rd Street, Galveston",POINT (-94.7913851 29.2989118),Curious,High,No,Low,indoor,Truck,NaN
3,Good Dough,-94.808531,29.290612,1,1,68.0,bakery,4.6,70,35,70,"1508 39th Street, Galveston",POINT (-94.808531 29.2906119),Curious,High,No,Med,indoor,Truck,NaN
4,Porch Cafe,-94.750192,29.319581,1,1,332.0,restaurant,4.1,300,NaN,100,"1625 East Beach Drive #2A, Galveston",POINT (-94.7501916 29.3195806),Curious,High,No,Low,indoor,Truck,NaN
5,Coastal Grill,-94.789455,29.308158,1,1,814.0,restaurant,4.6,500,100,500,"1827 The Strand, Galveston",POINT (-94.789455 29.308158),Skeptical,High,No,Med,indoor,Truck,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,Gypsy Joynt,-94.831648,29.274172,2,1,622.0,restaurant,4.5,2000,1400,2000,"6105 Stewart Rd Galveston, TX 77551",POINT (-94.8316475822502 29.2741723877485),TBD,Med,Yes,Med,indoor,Truck,NaN
167,Café Canela,-94.830826,29.274424,1,1,412.0,restaurant,4.6,350,50,350,"6105 Stewart Road Suite C, Galveston",POINT (-94.8308258 29.2744235),Skeptical,Med,Yes,Low,indoor,Truck,NaN
168,Gino's Italian Restaurant and Pizzeria,-94.832217,29.275314,1,1,1056.0,restaurant,4.3,35,5,35,"6124 Stewart Road, Galveston",POINT (-94.832217 29.2753139),Skeptical,Med,Yes,Low,indoor,Truck,NaN
169,Vida Agave,-94.795515,29.301840,1,1,155.0,restaurant,4.7,70,10,70,"711 25th Street, Galveston",POINT (-94.795515 29.30184),Skeptical,Med,Yes,Low,indoor,Truck,NaN


In [18]:
#Drop off Only Route, changing Lat and Long columns to Coordinate for Gurobi 

def create_coordinates(row):
    return f'({row["Latitude"]}, {row["Longitude"]})'

# Apply the function to create a new 'coordinates' column
DropoffOnly['Coordinates'] = DropoffOnly.apply(create_coordinates, axis=1)

# Display the DataFrame with the new 'coordinates' column
len(DropoffOnly)

170

In [24]:
#Starting Model

#number of locations
n=170
locations = [*range(n)]

#number of vans 
K=2
vans = [*range(K)]

#number of points is coordinates
points = DropoffOnly['Coordinates'].tolist()

# Extract coordinates from strings and convert to floats
points = [[float(coord) for coord in point.strip('()').split(',')] for point in points]

# Dict of Euclidean Distance
time = {(i, j): math.sqrt(sum((points[i][k] - points[j][k]) ** 2 for k in range(2)))
        for i in locations for j in locations if i != j}

In [30]:
#adding variables from gurobi, these can be changed:
# x =1, if van  k  visits and goes directly from location  i to location j 
x = m.addVars(time.keys(), vans, vtype=GRB.BINARY, name='FromToBy')

# y = 1, if customer i is visited by van k
y = m.addVars(locations, vans, vtype=GRB.BINARY, name='visitBy')

# Number of vans used is a decision variable
z = m.addVars(vans, vtype=GRB.BINARY, name='used')

# Travel time per van
t = m.addVars(vans, ub=120, name='travelTime')

#travel time
s = m.addVar(name='maxTravelTime')

In [31]:
#adding contraits from gurobi, removing time constraint for now

# Travel time constraint
# Exclude the time to return to the depot

travelTime = m.addConstrs((gp.quicksum(time[i,j]*x[i,j,k] for i,j in time.keys() if j > 0) == t[k] for k in vans), 
                          name='travelTimeConstr' )

# Van utilization constraint
visitCustomer = m.addConstrs((y[i,k] <= z[k]  for k in vans for i in locations if i > 0), name='visitCustomer' )

# Visit all customers
visitAll = m.addConstrs((y.sum(i,'*') == 1 for i in locations if i > 0), name='visitAll' )

# Depot constraint
depotConstr = m.addConstrs((y[0, k] == z[k] for k in vans), name='depotConstr' )

# Arriving at a customer location constraint
ArriveConstr = m.addConstrs((x.sum('*',j,k) == y[j,k] for j,k in y.keys()), name='ArriveConstr' )

# Leaving a customer location constraint
LeaveConstr = m.addConstrs((x.sum(j,'*',k) == y[j,k] for j,k in y.keys()), name='LeaveConstr' )

breakSymm = m.addConstrs((y.sum('*',k-1) >= y.sum('*',k) for k in vans if k>0), name='breakSymm' )

In [32]:
m.ModelSense = GRB.MINIMIZE
m.setObjectiveN(z.sum(), 0, priority=1, name="Number of vans")
m.setObjectiveN(s, 1, priority=0, name="Travel time")

In [33]:
# Callback - use lazy constraints to eliminate sub-tours
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._x)
        selected = gp.tuplelist((i,j) for i, j, k in model._x.keys()
                                if vals[i, j, k] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < n: 
            for k in vans:
                model.cbLazy(gp.quicksum(model._x[i, j, k]
                                         for i, j in permutations(tour, 2))
                             <= len(tour)-1)


# Given a tuplelist of edges, find the shortest subtour not containing depot (0)
def subtour(edges):
    unvisited = list(range(1, n))
    cycle = range(n+1)  # initial length has 1 more city
    while unvisited:
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            if current != 0:
                unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j == 0 or j in unvisited]
        if 0 not in thiscycle and len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle

In [34]:
m.write('weekly_tote_distribution.lp')

# Run optimization engine
m._x = x
m.Params.LazyConstraints = 1
m.optimize(subtourelim)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-8279U CPU @ 2.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads


User-callback calls 6, time in user-callback 0.00 sec


GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

In [35]:
# Print optimal routes
for k in vans:
    route = gp.tuplelist((i,j) for i,j in time.keys() if x[i,j,k].X > 0.5)
    if route:
        i = 0
        print(f"Route for van {k}: {i}", end='')
        while True:
            i = route.select(i, '*')[0][1]
            print(f" -> {i}", end='')
            if i == 0:
                break
        print(f". Travel time: {round(t[k].X,2)} min")

print(f"Max travel time: {round(s.X,2)}")

AttributeError: Unable to retrieve attribute 'X'